In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

SuperStore Dataset: Summary of sales per customer per product ; below is structure of data:

| Index | Column Name  |
| :---- | :----------- |
| 0     | Row ID       |
| 1     | Order ID     |
| 2     | Order Date   |
| 3     | Customer ID  |
| 4     | Segment      |
| 5     | City         |
| 6     | State        |
| 7     | Region       |
| 8     | Product ID   |
| 9     | Category     |
| 10    | Sub-Category |
| 11    | Sales        |
| 12    | Quantity     |
| 13    | Discount     |
| 14    | Profit       |

In [1]:
salesTransactions = sc.textFile("file///C:/data/SuperStore/SalesTransactions.csv",4).filter(lambda x : x.split(",")[0] != 'Row ID')

In [2]:
salesTransactionsRDD = salesTransactions.map(lambda x: x.split(",")).map(lambda y: (str(y[6]), float(y[11]))).cache()

In [3]:
type(salesTransactionsRDD)
salesTransactionsRDD.getNumPartitions()

4

In [4]:
salesTransactionsRDD.take(5)

[('Texas', 29.472),
 ('Texas', 1.248),
 ('Texas', 9.708),
 ('Texas', 27.24),
 ('Illinois', 230.376)]

In [32]:
totalSalesByState = salesTransactionsRDD.repartition(5).reduceByKey(lambda x,y : x + y)

In [33]:
#totalSalesByState.toDebugString()
totalSalesByState.getNumPartitions()

5

In [34]:
type(totalSalesByState.take(5))

list

In [39]:
for x in totalSalesByState.take(5):
    print ("%s has sales of %11.2f",(x[0] , x[1]))

%s has sales of %11.2f ('Indiana', 5745.8099999999995)
%s has sales of %11.2f ('Texas', 14811.428)
%s has sales of %11.2f ('Washington', 17010.54)
%s has sales of %11.2f ('Kentucky', 11135.34)
%s has sales of %11.2f ('Massachusetts', 3667.9820000000004)


In [41]:
totalSalesByState.sortBy(lambda x: -x[1]).take(15)

[('New York', 49620.664),
 ('California', 48675.27500000001),
 ('North Carolina', 17119.797999999995),
 ('Washington', 17010.54),
 ('Texas', 14811.428),
 ('Pennsylvania', 11998.995),
 ('Kentucky', 11135.34),
 ('Delaware', 10756.85),
 ('Tennessee', 8308.827),
 ('Florida', 8193.964),
 ('Ohio', 7696.97),
 ('Illinois', 7531.537),
 ('Indiana', 5745.8099999999995),
 ('Arizona', 5696.645),
 ('Michigan', 5425.090000000001)]

In [42]:
TransCountByState = salesTransactionsRDD.countByKey().items()

In [43]:
type(TransCountByState)

dict_items

In [44]:
TransCountByState

dict_items([('Texas', 118), ('Illinois', 51), ('Minnesota', 15), ('Oregon', 11), ('New York', 152), ('North Carolina', 40), ('Arizona', 27), ('Missouri', 5), ('New Jersey', 18), ('California', 239), ('Michigan', 29), ('Ohio', 51), ('Pennsylvania', 72), ('Tennessee', 33), ('Rhode Island', 12), ('Washington', 71), ('Kentucky', 33), ('Massachusetts', 17), ('Florida', 46), ('Colorado', 29), ('Indiana', 19), ('Georgia', 5), ('Maryland', 12), ('Wisconsin', 17), ('New Mexico', 10), ('Mississippi', 7), ('Iowa', 4), ('Nebraska', 5), ('Nevada', 1), ('Delaware', 3), ('Oklahoma', 7), ('Virginia', 11), ('Vermont', 1), ('Connecticut', 7), ('South Carolina', 4), ('Arkansas', 11), ('Idaho', 4), ('Kansas', 3), ('Louisiana', 6), ('Alabama', 7), ('Utah', 2), ('North Dakota', 2), ('South Dakota', 1), ('West Virginia', 1)])

In [46]:
for (k,v) in TransCountByState:
    print(k,"-", v)

Texas - 118
Illinois - 51
Minnesota - 15
Oregon - 11
New York - 152
North Carolina - 40
Arizona - 27
Missouri - 5
New Jersey - 18
California - 239
Michigan - 29
Ohio - 51
Pennsylvania - 72
Tennessee - 33
Rhode Island - 12
Washington - 71
Kentucky - 33
Massachusetts - 17
Florida - 46
Colorado - 29
Indiana - 19
Georgia - 5
Maryland - 12
Wisconsin - 17
New Mexico - 10
Mississippi - 7
Iowa - 4
Nebraska - 5
Nevada - 1
Delaware - 3
Oklahoma - 7
Virginia - 11
Vermont - 1
Connecticut - 7
South Carolina - 4
Arkansas - 11
Idaho - 4
Kansas - 3
Louisiana - 6
Alabama - 7
Utah - 2
North Dakota - 2
South Dakota - 1
West Virginia - 1


In [47]:
statesRDD = salesTransactionsRDD.map(lambda x: x[0]).distinct()

In [48]:
type(statesRDD)

pyspark.rdd.PipelinedRDD

In [51]:
StateCount = statesRDD.count()

In [52]:
type(StateCount)

int

In [53]:
StateCount

44

In [62]:
#salesByCategory = salesTransactions.map(lambda x : x.split(",")).map(lambda n : (str(n[9]), (n[11]))).cache()
salesByCategory = salesTransactions.map(lambda x : x.split(",")).map(lambda n : (str(n[9]), float(n[11]))).cache()

In [59]:
salesByCategory.take(5)

[('Office Supplies', '29.472'),
 ('Office Supplies', '1.248'),
 ('Furniture', '9.708'),
 ('Office Supplies', '27.24'),
 ('Office Supplies', '230.376')]

In [63]:
TotalSalesByCategory = salesByCategory.reduceByKey(lambda prev , curr : prev + curr )

In [64]:
TotalSalesByCategory.take(5)

[('Technology', 104759.34599999999),
 ('Furniture', 90348.24999999997),
 ('Office Supplies', 84946.47099999999)]

In [65]:
TotalSalesByCategory.getNumPartitions()

4

In [67]:
TotalSalesByCategory.takeOrdered(5, key = lambda x: x[1])

[('Office Supplies', 84946.47099999999),
 ('Furniture', 90348.24999999997),
 ('Technology', 104759.34599999999)]

In [70]:
salesByCategory.takeSample(True, 5)
#len(rdd.takeSample(True, 20, 1))

[('Office Supplies', 348.56),
 ('Furniture', 19.76),
 ('Furniture', 63.686),
 ('Technology', 361.376),
 ('Office Supplies', 580.672)]

In [71]:
salesBySubCategory = salesTransactions.map(lambda x : x.split(",")).map(lambda n : (str(n[10]), float(n[11]))).cache()

In [72]:
#salesBySubCategory.takeSample(True, 5)

[('Phones', 359.97),
 ('Paper', 19.44),
 ('Bookcases', 723.92),
 ('Machines', 7999.98),
 ('Tables', 652.45),
 ('Phones', 31.176),
 ('Furnishings', 77.952),
 ('Furnishings', 19.76),
 ('Tables', 1504.52),
 ('Furnishings', 77.6)]

In [74]:
salesBySubCategorypart10 = salesBySubCategory.partitionBy(10)

MapPartitionsRDD[102] at mapPartitions at PythonRDD.scala:436

In [85]:
def printPart(iterator):
        for x in iterator:
                print(x)
        print("===========") 

In [79]:
salesBySubCategory.count() #foreachPartition(printPart)

1219

In [86]:
salesBySubCategory.foreachPartition(printPart)

###  ReduceByKey / CombineByKey / AggregateByKey

In [87]:
TotalSalesBySubCategory = salesBySubCategory.reduceByKey(lambda accum, sales : accum + sales )

In [93]:
sortedTotalSalesBySubCategory = TotalSalesBySubCategory.sortBy(lambda x: -x[1])

In [94]:
sortedTotalSalesBySubCategory.take(5)

[('Phones', 35223.833999999995),
 ('Chairs', 34197.11),
 ('Tables', 33794.707),
 ('Copiers', 30239.792),
 ('Storage', 28216.642)]

In [107]:
from datetime import datetime
import calendar

salesByDate = salesTransactions.map(lambda x : x.split(",")).map(lambda n : (datetime.strptime(n[2], '%m/%d/%Y'), float(n[11]))).cache()

In [102]:
salesByDate.takeSample(True, 20)

[(datetime.datetime(2017, 12, 28, 0, 0), 340.704),
 (datetime.datetime(2017, 10, 23, 0, 0), 637.896),
 (datetime.datetime(2017, 12, 4, 0, 0), 255.968),
 (datetime.datetime(2017, 10, 7, 0, 0), 79.92),
 (datetime.datetime(2017, 12, 30, 0, 0), 52.776),
 (datetime.datetime(2017, 12, 16, 0, 0), 97.184),
 (datetime.datetime(2017, 12, 1, 0, 0), 131.98),
 (datetime.datetime(2017, 12, 8, 0, 0), 13.248),
 (datetime.datetime(2017, 10, 5, 0, 0), 7.992),
 (datetime.datetime(2017, 12, 25, 0, 0), 39.312),
 (datetime.datetime(2017, 11, 5, 0, 0), 16.03),
 (datetime.datetime(2017, 11, 20, 0, 0), 283.92),
 (datetime.datetime(2017, 11, 18, 0, 0), 38.864),
 (datetime.datetime(2017, 12, 28, 0, 0), 7.968),
 (datetime.datetime(2017, 12, 2, 0, 0), 242.352),
 (datetime.datetime(2017, 12, 8, 0, 0), 215.544),
 (datetime.datetime(2017, 12, 3, 0, 0), 13.592),
 (datetime.datetime(2017, 11, 20, 0, 0), 22.0),
 (datetime.datetime(2017, 10, 3, 0, 0), 83.92),
 (datetime.datetime(2017, 11, 2, 0, 0), 590.352)]

In [113]:
# find the sales average by day
salesByWeekDay = salesTransactions.map(lambda x : x.split(",")).map(lambda n : (calendar.day_name[datetime.strptime(n[2], '%m/%d/%Y').weekday()], float(n[11]))).cache()

In [112]:
#salesByWeekDay.takeSample(True, 20)
import calendar
list(calendar.day_name)

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [115]:
salesByWeekDay.takeSample(True, 10)

[('Sunday', 25.06),
 ('Saturday', 169.68),
 ('Monday', 23.92),
 ('Sunday', 27.46),
 ('Thursday', 48.86),
 ('Saturday', 35.48),
 ('Saturday', 18.336),
 ('Saturday', 1399.98),
 ('Thursday', 63.56),
 ('Friday', 187.76)]

In [121]:
salesSumCountRDD = salesByWeekDay.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1])).take(4)

In [119]:
#averageByKey = salesSumCountRDD.map(lambda (label, (value_sum, count)): (label, value_sum / count))

SyntaxError: invalid syntax (<ipython-input-119-1fb916187485>, line 1)

In [124]:
salesSumCountRDD

[('Thursday', (50313.729999999996, 203)),
 ('Saturday', (44369.536, 210)),
 ('Sunday', (51060.70600000001, 199)),
 ('Tuesday', (20484.023, 94))]

In [125]:
type(salesSumCountRDD)

list

In [130]:
for x in salesSumCountRDD:
    print(x[0], x[1][0]/x[1][1])

Thursday 247.85088669950736
Saturday 211.28350476190477
Sunday 256.58646231155785
Tuesday 217.91513829787235


### The combineByKey Method

In order to aggregate an RDD’s elements in parallel, Spark’s `combineByKey` method requires three functions:

- `createCombiner`
- `mergeValue`
- `mergeCombiner`


In [131]:
data = sc.parallelize( [(0, 2), (0, 4), (1, 0), (1, 10), (1, 20)] )

In [132]:
sumCount = data.combineByKey(lambda value: (value, 1),
                                 lambda x, value: (x[0] + value, x[1] + 1),
                                 lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [146]:
data.reduceByKey(lambda x,y : x + y).collect()

[(0, 6.0), (1, 30.0)]

In [157]:
data.aggregateByKey((0),lambda x,y : x + y, lambda x,y : (x[0] + y[0], x[1] + y[1])).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 164.0 failed 1 times, most recent failure: Lost task 1.0 in stage 164.0 (TID 450, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "c:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "c:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "c:\spark\python\pyspark\rdd.py", line 346, in func
    return f(iterator)
  File "c:\spark\python\pyspark\rdd.py", line 1850, in _mergeCombiners
    merger.mergeCombiners(iterator)
  File "c:\spark\python\lib\pyspark.zip\pyspark\shuffle.py", line 272, in mergeCombiners
    d[k] = comb(d[k], v) if k in d else v
  File "<ipython-input-157-a54440560228>", line 1, in <lambda>
TypeError: 'float' object is not subscriptable

	at org.apache.spark.api.pyth

In [151]:
for x in data.groupByKey().collect():
    print(x[1])

In [149]:
data.groupByKey().collect()

[(0, <pyspark.resultiterable.ResultIterable at 0x2571c4d2e10>),
 (1, <pyspark.resultiterable.ResultIterable at 0x2571c4d2208>)]